Relier la borne de tension de sortie Vout du capteur de pression à la borne de mesure EA0 de la carte Arduino.

Le programme à envoyer à Arduino est acquisition_continue.ino

On commence par importer les bibliothèques nécessaires. Attention : NE PAS EFFACER !

In [ ]:
import piplite; await piplite.install(['scipy', 'lmfit', 'ipympl']); from pylab import *; from Import_bibliotheques import *
%run Arduino.ipynb
connexion_arduino()

In [ ]:
nom = "variable"    # Donner entre les guillemets un nom pertinent à la variable dont dépend la pression

Acquisition des valeurs.  
Entrer une nouvelle valeur de la variable pour réaliser une nouvelle mesure. Pour arrêter, valider sans entrer de valeur.

In [ ]:
valeurs = nom+",P"
# L'étalonnage du capteur est du type P = coef * Vout + décalage ; les valeurs suivantes sont celles données par le constructeur pour le capteur de pression MPX4250AP
coef = 5e4
decalage = 1e4
while 1 :
    try :
        variable = float(input("Entrer la valeur du paramètre (entrer une ligne vide pour arrêter) : "))
        valeurs += f"\n{variable},"
        # Réinitialisation de l'Arduino et attente de la première ligne valide
        reset_arduino()
        tension = mesure_unique()
        
        pression = coef * tension + decalage
        print(f"pression mesurée : {pression:.2f} Pa")
        valeurs += f"{pression}"
    except :
        ser.close()
        
        fichier = open("pression.csv", 'w')
        fichier.write(valeurs)
        fichier.close()
        break

Importation de tableaux enregistrés localement en fichiers csv :

In [ ]:
tableurVersVariables('pression.csv')

Tracé de graphique(s) à partir des tableaux précédents :

In [ ]:
abscisse = nom    # choix de la grandeur en abscisse, pour le graphique et pour le modèle
ordonnee = "P"    # choix de la grandeur en ordonnée, pour le graphique et pour le modèle
equation = "a * x + b"    # expression de l'équation correspondant au modèle recherché ; la variable peut être x ou le nom donné en abscisse
debut = None    # indice du premier élément à prendre en compte pour le modèle (None : le premier de la liste)
fin = None    # indice du dernier élément à prendre en compte pour le modèle (None : le dernier de la liste)
debutCourbe = None    # valeur de l'abscisse où débute le tracé du modèle (None : la plus petite valeur de l'abscisse)
finCourbe = None    # valeur de l'abscisse où se termine le tracé du modèle (None : la plus grande valeur de l'abscisse)
contraintes = [['a', 1, True, -inf, inf]]    # ['nomDuParametre', valeurInitiale, variabilité, min, max]   à remplir pour chaque paramètre du modèle si la recherche automatique ne trouve pas de valeurs cohérentes

subplot(1,1,1)    # création d'une fenêtre de graphique : lignes, colonnes, numéro
xMod, yMod, expression, valeurs, abscisse, ordonnee, modele, parametres = Calcul_modele(abscisse, ordonnee, equation, debut, fin, debutCourbe, finCourbe, contraintes)    # Calcul des valeurs du modèle, ne pas modifier

# Tracé des points expérimentaux :
plot(abscisse, ordonnee, '+', label="")    # abscisse, ordonnée, options : + o . , s points   - -- -. : lignes  b g r y m c k w couleurs ; label est la légende associée à la courbe

# Tracé du modèle :
plot(xMod, yMod, '-', label=f"Modèle :    {expression} \n {valeurs}" )    # trace la courbe du modèle

#axis('equal')    # à décommenter si on veut un repère orthonormé
grid()    # ajoute une grille
legend()    # ajoute la légende au graphique
title("Mon titre")    # mettre le titre désiré entre les guillemets
xlabel("")    # mettre le nom de l'axe des abscisses entre les guillemets
ylabel("");    # mettre le nom de l'axe des ordonnées entre les guillemets. Le point-virgule final évite d'afficher un texte récapitulatif.

Calcul de valeurs particulières :